In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
from dataclasses import dataclass
import torch.nn as nn
from torch.nn import functional as F
import math

In [39]:
import tiktoken, time, inspect

Sources
- https://www.youtube.com/watch?v=l8pRSuU81PU&t=1839s&ab_channel=AndrejKarpathy

In [4]:
@dataclass
class GPTConfig:
    context_size: int = 1024
    vocab_size: int = 50257
    n_layer: int = 12
    n_head: int = 12
    n_embd: int = 768

In [5]:
class MultiHeadAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0

        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        self.c_proj.GPT_SCALE_INIT = 1
        self.n_head = config.n_head
        self.n_embd = config.n_embd

        self.register_buffer('bias', torch.tril(torch.ones(config.context_size, config.context_size))
                                    .view(1, 1, config.context_size, config.context_size))

    def forward(self, x):
        b,t,c = x.size()
        qkv = self.c_attn(x)
        q,k,v = qkv.split(self.n_embd, dim=2)
        q = q.view(b,t, self.n_head, c // self.n_head).transpose(1,2)
        k = k.view(b,t, self.n_head, c // self.n_head).transpose(1,2)
        v = v.view(b,t, self.n_head, c // self.n_head).transpose(1,2)

        #att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        #att = att.masked_fill(self.bias[:,:,:t,:t] == 0, float('-inf'))
        #att = F.softmax(att, dim=-1)
        #y = att @ v
        y = F.scaled_dot_product_attention(q, k, v, is_causal=True)
        y = y.transpose(1,2).contiguous().view(b,t,c)
        y = self.c_proj(y)
        return y

In [6]:
class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()

        self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd)
        self.gelu = nn.GELU(approximate='tanh')
        self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd)
        self.c_proj.GPT_SCALE_INIT = 1

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x

In [7]:
class Block(nn.Module):
    def __init__(self, config):
        super().__init__()

        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = MultiHeadAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

In [29]:
class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd), # weight token embedding
            wpe = nn.Embedding(config.context_size, config.n_embd), # weight position embedding
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = nn.LayerNorm(config.n_embd),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        # weight sharing scheme
        self.transformer.wte.weight = self.lm_head.weight

        # initialize parameters
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            std = 0.02
            if hasattr(module,'GPT_SCALE_INIT'):
                std *= (2 * self.config.n_layer) ** -0.5 # number of residual adds * number 
                # of layers 1/sqrt to clip scaling for each projection layer
            torch.nn.init.normal_(module.weight, mean=0.0, std=std)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
            elif isinstance(module, nn.Embedding):
                torch.nn.init.normal_(module.weight, mean=0.0, std=std)

    def forward(self, idx, targets=None):
        b, t = idx.size()
        assert t <= self.config.context_size, f"sequence too long"
        pos = torch.arange(0, t, dtype=torch.long, device=idx.device)
        pos_emb = self.transformer.wpe(pos)
        tok_emb = self.transformer.wte(idx)
        x = tok_emb + pos_emb
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)
        logits = self.lm_head(x)
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        return logits, loss

    def configure_optimizer(self, weight_decay, lr, device):
        param_dict = {pn: p for pn, p in self.named_parameters()}
        param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}

        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0}
        ]
        num_decay_params = sum(p.numel() for p in decay_params)
        num_nodecay_params = sum(p.numel() for p in nodecay_params)
        print(f'num decay param tensors: {len(decay_params)}, with {num_decay_params:,} parameters')
        print(f'num non-decayed param tensors: {len(nodecay_params)}, with {num_nodecay_params:,} parameters')
        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and 'cuda' in device
        print(f'using fused AdamW: {use_fused}')
        optimizer = torch.optim.AdamW(optim_groups, lr=lr, betas=(0.9, 0.95), eps=1e-8, fused=use_fused)
        return optimizer

    @classmethod
    def from_pretrained(cls, model_type):
        """Loads pretrained GPT-2 model weights from huggingface"""
        assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
        from transformers import GPT2LMHeadModel
        print("loading weights from pretrained gpt: %s" % model_type)

        # n_layer, n_head and n_embd are determined from model_type
        config_args = {
            'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
            'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024), # 350M params
            'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280), # 774M params
            'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600), # 1558M params
        }[model_type]
        config_args['vocab_size'] = 50257 # always 50257 for GPT model checkpoints
        config_args['context_size'] = 1024 # always 1024 for GPT model checkpoints
        # create a from-scratch initialized minGPT model
        config = GPTConfig(**config_args)
        model = GPT(config)
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')] # discard this mask / buffer, not a param

        # init a huggingface/transformers model
        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()

        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')] # ignore these, just a buffer
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')] # same, just the mask (buffer)
        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
        # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
        # this means that we have to transpose these weights when we import them
        assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                # special treatment for the Conv1D weights we need to transpose
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                # vanilla copy over the other parameters
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])

        return model

In [30]:
#tokens = enc.encode("Hello, I'm a language model,")
#tokens = torch.tensor(tokens, dtype=torch.long)
#tokens = tokens.unsqueeze(0).repeat(5, 1)
#x = tokens.to('cuda')

In [31]:
#torch.save(model.state_dict(), './data/hugging_face_weights')

In [32]:
#model2 = GPT(GPTConfig())
#model2.load_state_dict(torch.load('./data/hugging_face_weights', weights_only=True))
#model2.eval()
#model2.to('cuda')

In [33]:
class DataLoaderLite:
    def __init__(self, b, t):
        self.b = b
        self.t = t

        with open('./data/tiny_shakespeare.txt','r') as f: text = f.read()
        enc = tiktoken.get_encoding('gpt2')
        tokens = enc.encode(text)
        self.tokens = torch.tensor(tokens)
        print(f'loaded {len(self.tokens)} tokens')
        print(f' 1 epoch = {len(self.tokens) // (b*t)} batches')

        self.current_position = 0

    def next_batch(self):
        b, t = self.b, self.t
        buf = self.tokens[self.current_position : self.current_position+b*t+1]
        x = (buf[:-1]).view(b, t)
        y = (buf[1:]).view(b, t)
        self.current_position += b * t
        if self.current_position + (b * t + 1) > len(self.tokens):
            self.current_position = 0
        return x, y

In [34]:
train_loader = DataLoaderLite(4, 1024)
torch.set_float32_matmul_precision('high')

loaded 338024 tokens
 1 epoch = 82 batches


In [35]:
epochs = 50
max_lr=6e-4
min_lr = max_lr * 0.1
warmup_steps = 10
max_steps = 50

In [36]:
def cosine_lr_scheduler(it):
    if it < warmup_steps:
        return max_lr * (it+1) / warmup_steps
    if it > max_steps:
        return min_lr
    decay_ratio = (it - warmup_steps) / (max_steps - warmup_steps)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))
    return min_lr + coeff * (max_lr - min_lr)

In [37]:
model = GPT(GPTConfig(vocab_size=50304))
model.to('cuda')
model = torch.compile(model)

In [42]:
#optimizer = torch.optim.AdamW(model.parameters(), lr=max_lr)
optimizer = model.configure_optimizer(weight_decay=0.1, lr=6e-4, device='cuda')
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs, min_lr)

num decay param tensors: 50, with 124,354,560 parameters
num non-decayed param tensors: 98, with 121,344 parameters
using fused AdamW: True


default loss should be negative log likeliehood aka -ln(1/50257)

In [44]:
for i in range(epochs):
    t0 = time.time()
    x, y = train_loader.next_batch()
    x, y = x.to('cuda'), y.to('cuda')
    optimizer.zero_grad()
    with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
        logits, loss = model(x, y)
    loss.backward()
    norm = torch.nn.utils.clip_grad_norm_(model.parameters(),1.0)
    #lr = cosine_lr_scheduler(i)
    #for param_group in optimizer.param_groups:
    #    param_group['lr'] = lr
    optimizer.step()
    scheduler.step()
    torch.cuda.synchronize()
    t1 = time.time()
    dt = (t1 - t0) * 1000
    tokens_per_sec = (train_loader.b * train_loader.t) / (t1-t0)
    print(f'step {i}, loss: {loss.item()}, dt: {dt:.2f}ms, tok/sec: {tokens_per_sec:.2f}, norm: {norm},')

step 0, loss: 9.652811050415039, dt: 303.45ms, tok/sec: 13498.02, norm: 7.115577220916748,
step 1, loss: 9.492239952087402, dt: 100.99ms, tok/sec: 40558.26, norm: 4.09261417388916,
step 2, loss: 8.447969436645508, dt: 91.24ms, tok/sec: 44891.69, norm: 3.7570278644561768,
step 3, loss: 7.772185325622559, dt: 93.40ms, tok/sec: 43854.50, norm: 2.0779974460601807,
step 4, loss: 7.569711685180664, dt: 91.64ms, tok/sec: 44695.36, norm: 4.965698719024658,
step 5, loss: 7.247500896453857, dt: 90.24ms, tok/sec: 45391.03, norm: 5.085233211517334,
step 6, loss: 7.039894104003906, dt: 93.21ms, tok/sec: 43945.93, norm: 2.702420711517334,
step 7, loss: 6.715356826782227, dt: 91.33ms, tok/sec: 44848.44, norm: 1.4883853197097778,
step 8, loss: 6.77267599105835, dt: 90.48ms, tok/sec: 45271.54, norm: 1.6959059238433838,
step 9, loss: 6.926285266876221, dt: 92.80ms, tok/sec: 44138.32, norm: 1.954379677772522,
step 10, loss: 7.197279930114746, dt: 91.23ms, tok/sec: 44896.15, norm: 2.11727237701416,
step 1

### speedups

- matmul precision
- change type to bfloat16
- torch.compile
- deal with "ugly" numbers
- switch to flash attention

### hyper parameters
- gradient clipping
- cosine decay learning rate scheduler with warmup
- optimizer weight decay